## LCEL Tinkering

> https://python.langchain.com/docs/expression_language/get_started

---

In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

True

In [3]:
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_community.vectorstores import DocArrayInMemorySearch

In [5]:
chat_gpt4 = AzureChatOpenAI(
        deployment_name="gpt-4-32k",
        model="gpt-4-32k",
        openai_api_type="azure",
        azure_endpoint=os.environ['AZURE_API_BASE'],
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version=os.environ['AZURE_API_VERSION'],
        max_retries=2,
        temperature=0,
    )
embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_type='azure',
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION']
)

In [6]:
gpt_35_turbo = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    model="gpt-35-turbo",
    openai_api_type="azure",
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION'],
    max_retries=2,
    temperature=1,
)

In [7]:
gpt_35_turbo_instruct = AzureOpenAI(
    deployment_name="gpt-35-turbo-instruct",
    model="gpt-35-turbo-instruct",
    openai_api_type="azure",
    azure_endpoint=os.environ['AZURE_API_BASE'],
    openai_api_key=os.environ['AZURE_API_KEY'],
    openai_api_version=os.environ['AZURE_API_VERSION'],
    max_retries=2,
    temperature=1,
)

## Basic example: prompt + model + output parser

In [8]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = gpt_35_turbo
output_parser = StrOutputParser()

chain = prompt | model | output_parser

user_input = {"topic": "cricket"}

chain.invoke(user_input)

'Why did the cricket team go to the bank?\n\nTo get their cricket-credit!'

The `|` symbol is similar to a unix pipe operator, which chains together the different components feeds the output from one component as input into the next component.

In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let’s take a look at each component individually to really understand what’s going on.

### 1. prompt

prompt is a `BasePromptTemplate`, which means it takes in a dictionary of template variables and produces a PromptValue. A PromptValue is a wrapper around a completed prompt that can be passed to either an LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input). It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [22]:
prompt_output = prompt.invoke(user_input)
prompt_output

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about cricket')])

### 2. Model

The PromptValue is then passed to model. In this case our model is a ChatModel, meaning it will output a BaseMessage.

In [24]:
model_output = model.invoke(prompt_output)

model_output

AIMessage(content='Why did the cricket team go to the bank? Because they wanted to open a savings account!')

In [25]:
type(model_output)

langchain_core.messages.ai.AIMessage

If our model was an LLM, it would output a string.


In [27]:
llm = gpt_35_turbo_instruct

llm.invoke(prompt_output)

'\n\nAI: Why did the cricket player bring a tiny net to the game? Because he heard there was going to be a lot of caught flies!'

### 3. Output parser



And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. 
The StrOutputParser specifically simple converts any input into a string.

In [28]:
output_parser.invoke(model_output)

'Why did the cricket team go to the bank? Because they wanted to open a savings account!'

### 4. Entire Pipeline

To follow the steps along:

- We pass in user input on the desired topic as {"topic": "ice cream"}
- The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a ChatMessage object.
- Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

## RAG Search Example